In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import openpyxl
import datetime
import re
import string
from bs4 import BeautifulSoup
import requests
import requests
import pytz
from datetime import datetime as dt
import gspread

from date_extractor import extract_dates


# Read from Google Sheet

In [18]:
'''

# Read name variations from google sheets

gc = gspread.service_account(filename="/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json")

sheet_name = 'Name_Variations'
sheet_id = '1H3qeiHF1PKzoMG1aIGThvMn3UoBfIHLdLYCXYKOpAbs'

url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
data = pd.read_csv(url)

data
'''

'\n\n# Read name variations from google sheets\n\ngc = gspread.service_account(filename="/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json")\n\nsheet_name = \'Name_Variations\'\nsheet_id = \'1H3qeiHF1PKzoMG1aIGThvMn3UoBfIHLdLYCXYKOpAbs\'\n\nurl = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"\ndata = pd.read_csv(url)\n\ndata\n'

In [148]:
# Read name variations from google sheets

#gc = gspread.service_account(filename="/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json")

#sheet_name = 'SAA Competitions'
#sheet_id = '1zYD7ZwCAce2puHlRazCSACrKi3JH_mOL1rNi0waTBhE'

#url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
#data = pd.read_csv(url)

#data

# Open the Google Sheet by its title
#spreadsheet = gc.open("1zYD7ZwCAce2puHlRazCSACrKi3JH_mOL1rNi0waTBhE")
spreadsheet = gc.open_by_key("1zYD7ZwCAce2puHlRazCSACrKi3JH_mOL1rNi0waTBhE") 


# Select a specific worksheet
worksheet = spreadsheet.worksheet("Competitions")

# Get all values from the worksheet as a list of lists
data = worksheet.get_all_values()

df = pd.DataFrame(data)


In [149]:
df.columns = df.iloc[0]

df

,Competition Name,Country,Competition Date,Athlete Name,Event,Result,Wind,Gender,Comments,Uploads
0,Competition Name,Country,Competition Date,Athlete Name,Event,Result,Wind,Gender,Comments,Uploads
1,"Queensland 10,000m Championships",Australia,09/08/2025,Shaun Goh,10000m,31:02.40,,Male,https://worldathletics.org/competition/calenda...,https://drive.google.com/file/d/196cOw1tYQUkRY...
2,"Queensland 10,000m Championships",Australia,09/08/2025,Vanessa Lee,10000m,36:15.67,,Female,https://worldathletics.org/competition/calenda...,https://drive.google.com/file/d/196cOw1tYQUkRY...
3,7th Tokai Sprint Games,Japan,17/08/2025,Calvin Quek,100m,10.72,0.4,Male,https://worldathletics.org/competition/calenda...,
4,7th Tokai Sprint Games,Japan,17/08/2025,Praharsh Ryan,100m,10.70,0.0,Male,https://worldathletics.org/competition/calenda...,
5,7th Tokai Sprint Games,Japan,17/08/2025,Praharsh Ryan,100m,10.70,0.1,Male,https://worldathletics.org/competition/calenda...,
6,2025 ICAAK Games,Japan,12/08/2025,Calvin Quek,400m Hurdles (0.914m),50.24,,Male,https://worldathletics.org/competition/calenda...,https://drive.google.com/file/d/13TsWlLosDFnOu...
7,20th Twilight Games,Japan,20/08/2025,Calvin Quek,400m Hurdles (0.914m),49.75,,Male,,


In [150]:
df_clean = df.drop([0, 4, 5, 7])


In [151]:
df_clean

,Competition Name,Country,Competition Date,Athlete Name,Event,Result,Wind,Gender,Comments,Uploads
1,"Queensland 10,000m Championships",Australia,09/08/2025,Shaun Goh,10000m,31:02.40,,Male,https://worldathletics.org/competition/calenda...,https://drive.google.com/file/d/196cOw1tYQUkRY...
2,"Queensland 10,000m Championships",Australia,09/08/2025,Vanessa Lee,10000m,36:15.67,,Female,https://worldathletics.org/competition/calenda...,https://drive.google.com/file/d/196cOw1tYQUkRY...
3,7th Tokai Sprint Games,Japan,17/08/2025,Calvin Quek,100m,10.72,0.4,Male,https://worldathletics.org/competition/calenda...,
6,2025 ICAAK Games,Japan,12/08/2025,Calvin Quek,400m Hurdles (0.914m),50.24,,Male,https://worldathletics.org/competition/calenda...,https://drive.google.com/file/d/13TsWlLosDFnOu...


In [152]:
df_clean = df_clean.rename(columns={'Competition Name': 'COMPETITION', 'Country': 'COUNTRY', 'Competition Date': 'DATE', 
                                    'Athlete Name': 'NAME', 'Event': 'EVENT', 'Result': 'RESULT', 'Wind': 'WIND', 'Gender': 'GENDER'
                                   , 'Comments': 'REMARKS', 'Uploads': 'SOURCE'})


In [153]:
df_clean

,COMPETITION,COUNTRY,DATE,NAME,EVENT,RESULT,WIND,GENDER,REMARKS,SOURCE
1,"Queensland 10,000m Championships",Australia,09/08/2025,Shaun Goh,10000m,31:02.40,,Male,https://worldathletics.org/competition/calenda...,https://drive.google.com/file/d/196cOw1tYQUkRY...
2,"Queensland 10,000m Championships",Australia,09/08/2025,Vanessa Lee,10000m,36:15.67,,Female,https://worldathletics.org/competition/calenda...,https://drive.google.com/file/d/196cOw1tYQUkRY...
3,7th Tokai Sprint Games,Japan,17/08/2025,Calvin Quek,100m,10.72,0.4,Male,https://worldathletics.org/competition/calenda...,
6,2025 ICAAK Games,Japan,12/08/2025,Calvin Quek,400m Hurdles (0.914m),50.24,,Male,https://worldathletics.org/competition/calenda...,https://drive.google.com/file/d/13TsWlLosDFnOu...


In [154]:
# Extract event class

def event_class(string):
    
    
    try:
        
        left=re.search(r'\(', string)
        right=re.search(r'\)', string)
        
        
        type = string[left.start()+1:right.start()]
        
                 
    except:
        
        type = ''
    
    return type
    

df_clean['EVENT_CLASS'] = df_clean['EVENT'].apply(event_class)


In [155]:
def strip_brackets(series: pd.Series) -> pd.Series:
    """
    Remove text in parentheses (and the parentheses) from a Pandas Series of strings.
    Example: '400m Hurdles (0.914m)' -> '400m Hurdles'
    """
    return series.str.replace(r"\s*\(.*?\)", "", regex=True).str.strip()

df_clean['EVENT'] = strip_brackets(df_clean['EVENT'])

In [159]:
df_clean[['EVENT', 'EVENT_CLASS']]

,EVENT,EVENT_CLASS
1,10000m,
2,10000m,
3,100m,
6,400m Hurdles,0.914m


In [160]:
df_clean

,COMPETITION,COUNTRY,DATE,NAME,EVENT,RESULT,WIND,GENDER,REMARKS,SOURCE,EVENT_CLASS
1,"Queensland 10,000m Championships",Australia,09/08/2025,Shaun Goh,10000m,31:02.40,,Male,https://worldathletics.org/competition/calenda...,https://drive.google.com/file/d/196cOw1tYQUkRY...,
2,"Queensland 10,000m Championships",Australia,09/08/2025,Vanessa Lee,10000m,36:15.67,,Female,https://worldathletics.org/competition/calenda...,https://drive.google.com/file/d/196cOw1tYQUkRY...,
3,7th Tokai Sprint Games,Japan,17/08/2025,Calvin Quek,100m,10.72,0.4,Male,https://worldathletics.org/competition/calenda...,,
6,2025 ICAAK Games,Japan,12/08/2025,Calvin Quek,400m Hurdles,50.24,,Male,https://worldathletics.org/competition/calenda...,https://drive.google.com/file/d/13TsWlLosDFnOu...,0.914m


In [161]:
# Convert from dd/mm/yyyy to YYYY-MM-DD

df_clean["DATE"] = pd.to_datetime(df_clean["DATE"], dayfirst=True, utc=True, errors="coerce")


In [162]:
df_clean['DATE']

1   2025-08-09 00:00:00+00:00
2   2025-08-09 00:00:00+00:00
3   2025-08-17 00:00:00+00:00
6   2025-08-12 00:00:00+00:00
Name: DATE, dtype: datetime64[ns, UTC]

In [163]:
# Remove special characters
    
for col in df_clean.columns:
    df_clean[col] = df_clean[col].astype(str)
    df_clean[col] = df_clean[col].str.replace('\xa0', ' ', regex=True)
    df_clean[col] = df_clean[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    df_clean[col] = df_clean[col].str.replace('\r', ' ', regex=True)
    df_clean[col] = df_clean[col].str.replace('\n', ' ', regex=True)
    df_clean[col] = df_clean[col].str.strip()


    


In [164]:
# Map general event category 

df_clean.loc[df_clean['EVENT'].str.contains(r'^100m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
df_clean.loc[df_clean['EVENT'].str.contains(r'^400m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
df_clean.loc[df_clean['EVENT'].str.contains(r'^600m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Mid'
df_clean.loc[df_clean['EVENT'].str.contains(r'^60m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
df_clean.loc[df_clean['EVENT'].str.contains(r'^200m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
df_clean.loc[df_clean['EVENT'].str.contains(r'50m', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
df_clean.loc[df_clean['EVENT'].str.contains(r'80m', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
df_clean.loc[df_clean['EVENT'].str.contains(r'300m', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
df_clean.loc[df_clean['EVENT'].str.contains(r'^3000m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Long'
df_clean.loc[df_clean['EVENT'].str.contains(r'Mile', regex=True, na=True), 'CATEGORY_EVENT'] = 'Mid'
df_clean.loc[df_clean['EVENT'].str.contains(r'^5000m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Long'
df_clean.loc[df_clean['EVENT'].str.contains(r'^2000m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Mid'
df_clean.loc[df_clean['EVENT'].str.contains(r'^1000m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Mid'
df_clean.loc[df_clean['EVENT'].str.contains(r'^800m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Mid'
df_clean.loc[df_clean['EVENT'].str.contains(r'^1500m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Mid'
df_clean.loc[df_clean['EVENT'].str.contains(r'10,000m|10000m|10km', regex=True, na=True), 'CATEGORY_EVENT'] = 'Long'
df_clean.loc[df_clean['EVENT'].str.contains(r'5km|5000m', regex=True, na=True), 'CATEGORY_EVENT'] = 'Long'
df_clean.loc[df_clean['EVENT'].str.contains(r'Cross Country', na=True), 'CATEGORY_EVENT'] = 'Cross Country'


# Override values

df_clean.loc[df_clean['EVENT'].str.contains(r'Discus', na=True), 'CATEGORY_EVENT'] = 'Throw'
df_clean.loc[df_clean['EVENT'].str.contains(r'Javelin', na=True), 'CATEGORY_EVENT'] = 'Throw'
df_clean.loc[df_clean['EVENT'].str.contains(r'Shot', na=True), 'CATEGORY_EVENT'] = 'Throw'
df_clean.loc[df_clean['EVENT'].str.contains(r'Vault', na=True), 'CATEGORY_EVENT'] = 'Jump'
df_clean.loc[df_clean['EVENT'].str.contains(r'Throw', na=True), 'CATEGORY_EVENT'] = 'Throw'
df_clean.loc[df_clean['EVENT'].str.contains(r'Jump', na=True), 'CATEGORY_EVENT'] = 'Jump'
df_clean.loc[df_clean['EVENT'].str.contains(r'Relay', na=True), 'CATEGORY_EVENT'] = 'Relay'
df_clean.loc[df_clean['EVENT'].str.contains(r'Steeple', na=True), 'CATEGORY_EVENT'] = 'Steeple'
df_clean.loc[df_clean['EVENT'].str.contains(r'Walk', na=True), 'CATEGORY_EVENT'] = 'Walk'
df_clean.loc[df_clean['EVENT'].str.contains(r'Hurdles', na=True), 'CATEGORY_EVENT'] = 'Hurdles'
df_clean.loc[df_clean['EVENT'].str.contains(r'Pentathlon', na=True), 'CATEGORY_EVENT'] = 'Pentathlon'
df_clean.loc[df_clean['EVENT'].str.contains(r'Heptathlon', na=True), 'CATEGORY_EVENT'] = 'Heptathlon'
df_clean.loc[df_clean['EVENT'].str.contains(r'Triathlon', na=True), 'CATEGORY_EVENT'] = 'Triathlon'
df_clean.loc[df_clean['EVENT'].str.contains(r'Decathlon', na=True), 'CATEGORY_EVENT'] = 'Decathlon'
df_clean.loc[df_clean['EVENT'].str.contains(r'Marathon', na=True), 'CATEGORY_EVENT'] = 'Marathon'
df_clean.loc[df_clean['EVENT'].str.contains(r'4 x 100m', na=True), 'CATEGORY_EVENT'] = 'Relay'
df_clean.loc[df_clean['EVENT'].str.contains(r'4 x 400m', na=True), 'CATEGORY_EVENT'] = 'Relay'

# Road race

df_clean.loc[df_clean['EVENT'].str.contains(r'Road|road', na=True), 'CATEGORY_EVENT'] = 'Road'


# Copy relay event dictionary names into NAME column

#df_clean['NAME'] = np.where(((df_clean['NAME'] == '') & ((df_clean['EVENT']=='4 x 100m')|(df_clean['EVENT']=='4 x 400m'))), df_clean['DICT_RESULTS'], df_clean['NAME'])


In [165]:
# Standardise Event Name

mask = df_clean['EVENT'].str.contains(r'^10000m$', na=True)
df_clean.loc[mask, 'EVENT'] = '10,000m'

mask = df_clean['EVENT'].str.contains(r'3000m Steeplechase|3000m S\/C', na=True)
df_clean.loc[mask, 'EVENT'] = '3000m Steeplechase'

mask = df_clean['EVENT'].str.contains(r'4 X 100m Relay|4x100m Relay', na=True)
df_clean.loc[mask, 'EVENT'] = '4 x 100m'

mask = df_clean['EVENT'].str.contains(r'4 X 400m Relay|4x400m Relay', na=True)
df_clean.loc[mask, 'EVENT'] = '4 x 400m'


In [166]:
# Convert to NEW SCHEMA

df_clean['LAST_NAME'] = ''
df_clean['FIRST_NAME'] = ''
df_clean['OTHER_NAME'] = ''
df_clean['TEAM'] = ''
df_clean['SEED'] = ''
df_clean['LANE'] = ''
df_clean['DIVISION'] = ''
df_clean['AGE'] = ''
df_clean['UNIQUE_ID'] = ''
df_clean['ATHLETE_ID'] = ''
df_clean['TIMESTAMP'] = ''
df_clean['TAG_ID'] = ''
df_clean['POINTS'] = ''
df_clean['GROUP'] = ''
df_clean['SESSION']=''
df_clean['DISTANCE']=''
df_clean['HEAT'] = ''
df_clean['LANE'] = ''
df_clean['QUALIFICATION']=''
df_clean['WIND']=''
df_clean['DICT_RESULTS'] = ''
df_clean['DOB'] = ''
df_clean['VENUE']=''
df_clean['SUB_EVENT']=''
df_clean['HOST_CITY']=''
df_clean['RX_TIME']=''

df_clean['NATIONALITY']='SGP'
df_clean['YEAR']='2025'
df_clean['REGION']='International'

df_clean = df_clean.reindex(columns= ['FIRST_NAME', 'LAST_NAME', 'OTHER_NAME', 'NAME', 'RANK', 'TAG_ID', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION',
                                        'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'NATIONALITY',
                                        'DICT_RESULTS', 'YEAR', 'DATE', 'COMPETITION', 'REGION', 'DOB', 'GROUP', 'CATEGORY_EVENT', 'ATHLETE_ID',
                                        'SOURCE', 'REMARKS', 'TIMESTAMP', 'VENUE', 'SUB_EVENT', 'SESSION', 'EVENT_CLASS', 'DISTANCE', 'HOST_CITY', 'RX_TIME'])


In [167]:
df_clean

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
1,,,,Shaun Goh,NaN,,,,31:02.40,,...,https://drive.google.com/file/d/196cOw1tYQUkRY...,https://worldathletics.org/competition/calenda...,,,,,,,,
2,,,,Vanessa Lee,NaN,,,,36:15.67,,...,https://drive.google.com/file/d/196cOw1tYQUkRY...,https://worldathletics.org/competition/calenda...,,,,,,,,
3,,,,Calvin Quek,NaN,,,,10.72,,...,,https://worldathletics.org/competition/calenda...,,,,,,,,
6,,,,Calvin Quek,NaN,,,,50.24,,...,https://drive.google.com/file/d/13TsWlLosDFnOu...,https://worldathletics.org/competition/calenda...,,,,,0.914m,,,


In [168]:
# Save to Excel

df_clean.to_excel("check.xlsx", index=False)

In [169]:
df = df_clean

# Check dataframe before upload into BQ

In [170]:
'''
# Check file before saving
import datetime


gender_pattern = 'Male|Female|Mixed'
year_pattern = '2025'
format_string = "%Y-%m-%d %H:%M:%S%z"


df['YEAR'] = df['YEAR'].astype(str)
df['DATE'] = df['DATE'].astype(str)
df['DOB'] = df['DOB'].astype(str)

stage_list = ['Finals', 'Semifinal', 'Heats', 'Qualification', '', ' ']

event_list = ['100m', '300m', '400m', '800m', '1500m', '1500m Walk', '3000m', '5000m', '10,000m', '80m Hurdles', '110m Hurdles', '100m Hurdles', '200m Hurdles', '400m Hurdles', 
              'High Jump', 'Discus', 'Discus Throw', 'Triple Jump', 'Pole Vault','Long Jump', 'Javelin', 'Javelin Throw', 'Shot Put', 'High Jump', '2000m SC', '3000m Walk', 
              '5000m Walk', 'Hammer Throw', '200m', '3000m SC', '4 x 100m Relay', '4 x 400m Relay'] 
category_list = ['Throw', 'Jump', 'Relay', 'Long', 'Mid', 'Sprint', 'Road', 'Cross Country', 'Steeple', 'Hurdles', 'Walk', 'Marathon', 'Triathlon', 'Decathlon', 'Pentathlon']

for index, row in df.iterrows():
        
    assert re.search(gender_pattern, row['GENDER']) is not None   # check format of gender input
    assert row['NAME'] != None
    #assert len(row['NAME'])>5
    assert row['DATE'] != None
    assert row['EVENT'] != None 
    assert row['CATEGORY_EVENT'] != None 
    assert row['COMPETITION'] != None 
    assert (row['REGION'] == 'Local' or row['REGION'] == 'International')
    assert row['DATE'] is not None
    assert row['CA2025-08-22 16:54:00+00:00TEGORY_EVENT'] in category_list
    assert len(row['DATE'])>3
    assert re.search('\)|\(', row['EVENT']) is None  # make sure there are no brackets in the event column
    assert row['NATIONALITY']=='SGP' # For Tilastopaja extractions
    assert re.search(year_pattern, row['YEAR']) is not None   # check format of year
    assert row['STAGE'] in stage_list 
    assert row['EVENT'] in event_list 
    assert re.search('Para|para', row['EVENT']) is None

    assert datetime.datetime.strptime(row['DATE'], format_string)

    
    # Check column locations before loading

    assert len(df.columns) == 40
    assert df.columns.get_loc('RESULT') == 8
    assert df.columns.get_loc('AGE') == 17
    assert df.columns.get_loc('WIND') == 12
    assert df.columns.get_loc('COMPETITION')==24
    
   

    # Set timestamp

    timezone = pytz.timezone('UTC')
    dt_ms = datetime.datetime.now().replace(tzinfo=timezone)
    df.loc[:,'TIMESTAMP'] = dt_ms.replace(second=0, microsecond=0)


print('PASSED')

df
'''

'\n# Check file before saving\nimport datetime\n\n\ngender_pattern = \'Male|Female|Mixed\'\nyear_pattern = \'2025\'\nformat_string = "%Y-%m-%d %H:%M:%S%z"\n\n\ndf[\'YEAR\'] = df[\'YEAR\'].astype(str)\ndf[\'DATE\'] = df[\'DATE\'].astype(str)\ndf[\'DOB\'] = df[\'DOB\'].astype(str)\n\nstage_list = [\'Finals\', \'Semifinal\', \'Heats\', \'Qualification\', \'\', \' \']\n\nevent_list = [\'100m\', \'300m\', \'400m\', \'800m\', \'1500m\', \'1500m Walk\', \'3000m\', \'5000m\', \'10,000m\', \'80m Hurdles\', \'110m Hurdles\', \'100m Hurdles\', \'200m Hurdles\', \'400m Hurdles\', \n              \'High Jump\', \'Discus\', \'Discus Throw\', \'Triple Jump\', \'Pole Vault\',\'Long Jump\', \'Javelin\', \'Javelin Throw\', \'Shot Put\', \'High Jump\', \'2000m SC\', \'3000m Walk\', \n              \'5000m Walk\', \'Hammer Throw\', \'200m\', \'3000m SC\', \'4 x 100m Relay\', \'4 x 400m Relay\'] \ncategory_list = [\'Throw\', \'Jump\', \'Relay\', \'Long\', \'Mid\', \'Sprint\', \'Road\', \'Cross Country\', \

In [171]:
import pandas as pd
import re
import datetime

gender_pattern = 'Male|Female|Mixed'
year_pattern = '2025'
format_string = "%Y-%m-%d %H:%M:%S%z"

stage_list = ['Finals', 'Semifinal', 'Heats', 'Qualification', 'Prelim', '']
event_list = [
    '60m', '60m Hurdles', '100m', '300m', '400m', '800m', '1500m', '1500m Walk', 
    '3000m', '5000m', '10,000m', '80m Hurdles', '110m Hurdles', '100m Hurdles', 
    '200m Hurdles', '300m Hurdles', '400m Hurdles', 'High Jump', 'Discus', 
    'Discus Throw', 'Triple Jump', 'Pole Vault', 'Long Jump', 'Javelin', 
    'Javelin Throw', 'Shot Put', 'High Jump', '2000m SC', '3000m Walk', 
    '5000m Walk', 'Hammer Throw', '200m', '3000m SC', '4 x 100m Relay', 
    '4 x 400m Relay', '1500m Racewalk', '3000m Racewalk', '5000m Racewalk', 'Mile Road'
]
category_list = [
    'Throw', 'Jump', 'Relay', 'Long', 'Mid', 'Sprint', 'Road', 
    'Cross Country', 'Steeple', 'Hurdles', 'Walk', 'Marathon', 
    'Triathlon', 'Decathlon', 'Pentathlon', 'Octathlon'
]

# ----------------------------
# Centralized cleaning step
# ----------------------------
# Convert to string where necessary
df['YEAR'] = df['YEAR'].astype(str)
df['DATE'] = df['DATE'].astype(str)
df['DOB'] = df['DOB'].astype(str)

# Normalize all missing/null values (empty, None, nan, NaN) → pd.NA
df = df.replace(["", "None", "none", "nan", "NaN"], pd.NA)

# Quick check of missing values
missing_summary = df.isna().sum()
print("Missing values per column:\n", missing_summary)

# ----------------------------
# Validation loop
# ----------------------------
for index, row in df.iterrows():
    # Generic null checks
    assert not pd.isna(row['NAME']), f"NAME missing at row {index}"
    assert not pd.isna(row['DATE']), f"DATE missing at row {index}"
    assert not pd.isna(row['EVENT']), f"EVENT missing at row {index}"
    assert not pd.isna(row['COMPETITION']), f"COMPETITION missing at row {index}"

    # Domain-specific validations
#    assert re.search(gender_pattern, row['GENDER']) is not None, f"Invalid GENDER at row {index}"
#    assert row['REGION'] in ['Local', 'International'], f"Invalid REGION at row {index}"
#    assert row['CATEGORY_EVENT'] in category_list, f"Invalid CATEGORY_EVENT at row {index}"
#    assert re.search(r'\)|\(', row['EVENT']) is None, f"Invalid EVENT (brackets) at row {index}"
#    assert row['NATIONALITY'] == 'SGP', f"Unexpected NATIONALITY at row {index}"
#    assert re.search(year_pattern, row['YEAR']) is not None, f"Invalid YEAR at row {index}"
#    assert row['STAGE'] in stage_list, f"Invalid STAGE at row {index}"
#    assert row['EVENT'] in event_list, f"Invalid EVENT at row {index}"

# Domain-specific validations with value reporting
    assert re.search(gender_pattern, row['GENDER']) is not None, (f"Invalid GENDER '{row['GENDER']}' at row {index}")

    assert row['REGION'] in ['Local', 'International'], (f"Invalid REGION '{row['REGION']}' at row {index}")

    assert row['CATEGORY_EVENT'] in category_list, (f"Invalid CATEGORY_EVENT '{row['CATEGORY_EVENT']}' at row {index}")

    assert re.search(r'\)|\(', row['EVENT']) is None, (f"Invalid EVENT (brackets) '{row['EVENT']}' at row {index}")

    assert row['NATIONALITY'] == 'SGP', (f"Unexpected NATIONALITY '{row['NATIONALITY']}' at row {index}")

    assert re.search(year_pattern, row['YEAR']) is not None, (f"Invalid YEAR '{row['YEAR']}' at row {index}")

    assert row['EVENT'] in event_list, (f"Invalid EVENT '{row['EVENT']}' at row {index}")

    # ---- Stage check (allow NaN / None / null)
    if pd.notna(row['STAGE']) and row['STAGE'] not in stage_list:
        raise ValueError(f"Invalid STAGE: {row['STAGE']}")

    
    # Date parsing check
    datetime.datetime.strptime(row['DATE'], format_string)

# Column structure validation
assert len(df.columns) == 40, "Unexpected number of columns"
assert df.columns.get_loc('RESULT') == 8, "RESULT column misplaced"
assert df.columns.get_loc('AGE') == 17, "AGE column misplaced"
assert df.columns.get_loc('WIND') == 12, "WIND column misplaced"
assert df.columns.get_loc('COMPETITION') == 24, "COMPETITION column misplaced"

print("✅ ALL CHECKS PASSED")

Missing values per column:
 0
FIRST_NAME        4
LAST_NAME         4
OTHER_NAME        4
NAME              0
RANK              4
TAG_ID            4
TEAM              4
SEED              4
RESULT            0
QUALIFICATION     4
HEAT              4
LANE              4
WIND              4
EVENT             0
DIVISION          4
STAGE             4
POINTS            4
AGE               4
GENDER            0
UNIQUE_ID         4
NATIONALITY       0
DICT_RESULTS      4
YEAR              0
DATE              0
COMPETITION       0
REGION            0
DOB               4
GROUP             4
CATEGORY_EVENT    0
ATHLETE_ID        4
SOURCE            1
REMARKS           0
TIMESTAMP         4
VENUE             4
SUB_EVENT         4
SESSION           4
EVENT_CLASS       3
DISTANCE          4
HOST_CITY         4
RX_TIME           4
dtype: int64
✅ ALL CHECKS PASSED


In [172]:
# Set timestamp

import datetime


timezone = pytz.timezone('UTC')
dt_ms = datetime.datetime.now().replace(tzinfo=timezone)
df.loc[:,'TIMESTAMP'] = dt_ms.replace(second=0, microsecond=0)


In [173]:
df

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
1,<NA>,<NA>,<NA>,Shaun Goh,NaN,<NA>,<NA>,<NA>,31:02.40,<NA>,...,https://drive.google.com/file/d/196cOw1tYQUkRY...,https://worldathletics.org/competition/calenda...,2025-08-23 00:18:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,<NA>,<NA>,<NA>,Vanessa Lee,NaN,<NA>,<NA>,<NA>,36:15.67,<NA>,...,https://drive.google.com/file/d/196cOw1tYQUkRY...,https://worldathletics.org/competition/calenda...,2025-08-23 00:18:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,<NA>,<NA>,<NA>,Calvin Quek,NaN,<NA>,<NA>,<NA>,10.72,<NA>,...,<NA>,https://worldathletics.org/competition/calenda...,2025-08-23 00:18:00+00:00,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,<NA>,<NA>,<NA>,Calvin Quek,NaN,<NA>,<NA>,<NA>,50.24,<NA>,...,https://drive.google.com/file/d/13TsWlLosDFnOu...,https://worldathletics.org/competition/calenda...,2025-08-23 00:18:00+00:00,<NA>,<NA>,<NA>,0.914m,<NA>,<NA>,<NA>


In [165]:
from google.cloud import bigquery
from google.cloud.bigquery import SchemaField


# Load BQ Modified Schema for TEST

modified_schema = [
            bigquery.SchemaField("FIRST_NAME", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("LAST_NAME", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("OTHER_NAME", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("NAME", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("RANK", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("TAG_ID", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("TEAM", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("SEED", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("RESULT", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("QUALIFICATION", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("HEAT", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("LANE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("WIND", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("EVENT", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("DIVISION", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("STAGE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("POINTS", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("AGE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("GENDER", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("UNIQUE_ID", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("NATIONALITY", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("DICT_RESULTS", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("YEAR", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("DATE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("COMPETITION", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("REGION", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("DOB", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("GROUP", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("CATEGORY_EVENT", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("ATHLETE_ID", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("SOURCE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("REMARKS", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("TIMESTAMP", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("VENUE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("SUB_EVENT", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("SESSION", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("EVENT_CLASS", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("DISTANCE", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("HOST_CITY", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("RX_TIME", "STRING", mode="NULLABLE")


]

In [166]:
df = df.astype(str)

In [170]:
# Write and append to TEST table in BigQuery directly with schema

client = bigquery.Client()

job_config = bigquery.LoadJobConfig(
    schema=modified_schema,
    write_disposition="WRITE_APPEND",  # or WRITE_APPEND, WRITE_EMPTY
)

job = client.load_table_from_dataframe(df, 'saa-analytics.results.PRODUCTION', job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=saa-analytics, location=US, id=89369ff5-7bef-4490-a615-7b0974fe4485>

In [171]:
# Check that loading was successful

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT *
FROM `saa-analytics.results.PRODUCTION`
WHERE TIMESTAMP='2025-08-22 16:54:00+00:00'

"""

extract = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)


extract

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
0,nan,nan,nan,Ryan Praharsh,5.0,nan,nan,nan,10.70,nan,...,Tilastopaja - 7th Tokai Sprint Games - Aug 25...,PB,2025-08-22 16:54:00+00:00,Lemon Gas Stadium,nan,nan,nan,nan,Hiratsuka,nan
1,nan,nan,nan,Ryan Praharsh,8.0,nan,nan,nan,10.70,nan,...,Tilastopaja - 7th Tokai Sprint Games - Aug 25...,PB,2025-08-22 16:54:00+00:00,Lemon Gas Stadium,nan,nan,nan,nan,Hiratsuka,nan
2,nan,nan,nan,Kerstin Ong Jing Rong,1.0,nan,nan,nan,12.61,nan,...,Tilastopaja - 101st WASEDA & KEIO Athletic Mee...,nan,2025-08-22 16:54:00+00:00,Keio Univ. Hiyoshi Athletics Field,nan,nan,nan,nan,Yokohama,nan
3,nan,nan,nan,Kerstin Ong Jing Rong,1.0,nan,nan,nan,14.22,nan,...,Tilastopaja - 101st WASEDA & KEIO Athletic Mee...,nan,2025-08-22 16:54:00+00:00,Keio Univ. Hiyoshi Athletics Field,nan,nan,nan,nan,Yokohama,nan
4,nan,nan,nan,Romaine Soh,14.0,nan,nan,nan,4:44.67,nan,...,Tilastopaja - BMC Record Breaker Races - Aug 2...,nan,2025-08-22 16:54:00+00:00,Tooting Bec Athletics Track,nan,nan,nan,nan,Tooting Bec,nan
5,nan,nan,nan,Jing Heng Ng,9.0,nan,nan,nan,4:10.09,nan,...,Tilastopaja - Watford Harriers Open Graded Mee...,SB,2025-08-22 16:54:00+00:00,Woodside Stadium,nan,nan,nan,nan,Watford,nan
6,nan,nan,nan,Rui Yong Soh,nan,nan,nan,nan,DNF,nan,...,Tilastopaja - Meeting voor Mon - Aug 25.html,nan,2025-08-22 16:54:00+00:00,AtletiekArena Gaston Roelants Kessel-Lo / Hal 5,nan,nan,nan,nan,Leuven,nan
7,nan,nan,nan,Jun Jie Calvin Quek,1.0,nan,nan,nan,50.80,nan,...,Tilastopaja - National Track and Field Regiona...,nan,2025-08-22 16:54:00+00:00,Liaoning Athletics Management Center,nan,nan,nan,nan,Dalian,nan
8,nan,nan,nan,Jun Jie Calvin Quek,1.0,nan,nan,nan,50.82,nan,...,Tilastopaja - National Track and Field Regiona...,nan,2025-08-22 16:54:00+00:00,Liaoning Athletics Management Center,nan,nan,nan,nan,Dalian,nan
9,nan,nan,nan,Jun Jie Calvin Quek,1.0,nan,nan,nan,49.75,nan,...,Tilastopaja - The 20th Twilight Games - Aug 25...,PB,2025-08-22 16:54:00+00:00,International Stadium,nan,nan,nan,nan,Yokohama,nan


In [38]:
extract['DATE']

0     2025-08-11 00:00:00+00:00
1     2025-08-11 00:00:00+00:00
2     2025-08-11 00:00:00+00:00
3     2025-08-11 00:00:00+00:00
4     2025-08-09 00:00:00+00:00
5     2025-08-02 00:00:00+00:00
6     2025-08-09 00:00:00+00:00
7     2025-08-09 00:00:00+00:00
8     2025-08-10 00:00:00+00:00
9     2025-08-10 00:00:00+00:00
10    2025-08-09 00:00:00+00:00
11    2025-08-09 00:00:00+00:00
12    2025-08-09 00:00:00+00:00
Name: DATE, dtype: object

In [175]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Singapore Athletes Events/2025/Aug/batch 2/')

df.to_excel("SGP_athletes_Aug_2025_2nd_batch.xlsx")

# Read and process manually cleaned data

In [753]:
# read manually cleaned data

SGP_athletes = pd.read_csv("SGP_athletes_cleaned.csv")

In [754]:
SGP_athletes

,Unnamed: 0,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,...,GENDER,STAGE,HEAT,WIND,DOB,REMARKS,RX_TIME,DICT_RESULT,SOURCE,REGION
0,102,4.0,Jun Jie Calvin Quek,SGP,53.19,NaN,UAE Athletics Grand Prix,2024,,400m Hurdles,...,Male,,,,26 Feb 96,« »,,[],Tilastopaja - UAE Athletics Grand Prix - May ...,International
1,31,2.0,Chui Ling Goh,SGP,17:33.73,NaN,11. Münchner Abendsportfest /MUC HochschulMS,2024,,5000m,...,Female,,,,27 Nov 92,« »,Mixed competition,[],Tilastopaja - 11. Münchner Abendsportfest :MU...,International
2,121,4.0,Chen Xiang Ang,SGP,8.01,NaN,11th Asian Indoor Athletics Championships,2024,17 February,60m Hurdles,...,Male,Heats,Heat 1,,3 Jul 94,SB «,,[],Tilastopaja - 11th Asian Indoor Athletics Cham...,International
3,112,8.0,Jun Jie Calvin Quek,SGP,52.05,NaN,11th Kinami Michitaka Memorial Athletics Meet,2024,,400m Hurdles,...,Male,,,,26 Feb 96,SB «,,[],Tilastopaja - 11th Kinami Michitaka Memorial A...,International
4,482,4.0,James Ethan Kai Meng Ang,SGP,52.55,NaN,11th Para Athletics World Championships,2024,22 May,400m,...,Male,Heats,Heat 2,,26 May 01,« »,0.187,[],Tilastopaja - 11th Para Athletics World Champi...,International
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,127,3.0,Jun Jie Calvin Quek,SGP,56.95,NaN,XXXIII Qosanov Memorial,2024,22 June,400m Hurdles,...,Male,Heats,Heat 2,,26 Feb 96,« »,,[],Tilastopaja - XXXIII Qosanov Memorial - Jun 24...,International
359,148,4.0,Kangli Emery Conrad,SGP,7.22,NaN,XXXIII Qosanov Memorial,2024,22 June,Long Jump,...,Male,NaN,NaN,NaN,26 Apr 06,« »,,[],Tilastopaja - XXXIII Qosanov Memorial - Jun 24...,International
360,320,5.0,Tia Louise Rozario,SGP,5.71,NaN,XXXIII Qosanov Memorial,2024,22 June,Long Jump,...,Female,NaN,NaN,NaN,14 Oct 00,« »,,[],Tilastopaja - XXXIII Qosanov Memorial - Jun 24...,International
361,330,5.0,Tia Louise Rozario,SGP,12.45,NaN,XXXIII Qosanov Memorial,2024,23 June,Triple Jump,...,Female,NaN,NaN,NaN,14 Oct 00,« »,,[],Tilastopaja - XXXIII Qosanov Memorial - Jun 24...,International


In [755]:
# Map general event category 

SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'100m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'400m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'600m', na=True), 'CATEGORY_EVENT'] = 'Mid'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'60m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'200m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'50m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'80m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'300m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'3000m', na=True), 'CATEGORY_EVENT'] = 'Long'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Mile', na=True), 'CATEGORY_EVENT'] = 'Mid'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'5000m', na=True), 'CATEGORY_EVENT'] = 'Long'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'2000m', na=True), 'CATEGORY_EVENT'] = 'Mid'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'800m', na=True), 'CATEGORY_EVENT'] = 'Mid'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'1500m', na=True), 'CATEGORY_EVENT'] = 'Mid'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'10,000m', na=True), 'CATEGORY_EVENT'] = 'Long'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'5km', na=True), 'CATEGORY_EVENT'] = 'Long'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'10km', na=True), 'CATEGORY_EVENT'] = 'Long'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Cross Country', na=True), 'CATEGORY_EVENT'] = 'Cross Country'


# Override values

SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Discus', na=True), 'CATEGORY_EVENT'] = 'Throw'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Javelin', na=True), 'CATEGORY_EVENT'] = 'Throw'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Shot', na=True), 'CATEGORY_EVENT'] = 'Throw'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Vault', na=True), 'CATEGORY_EVENT'] = 'Jump'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Throw', na=True), 'CATEGORY_EVENT'] = 'Throw'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Jump', na=True), 'CATEGORY_EVENT'] = 'Jump'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Relay', na=True), 'CATEGORY_EVENT'] = 'Relay'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Steeple', na=True), 'CATEGORY_EVENT'] = 'Steeple'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Walk', na=True), 'CATEGORY_EVENT'] = 'Walk'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Hurdles', na=True), 'CATEGORY_EVENT'] = 'Hurdles'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Pentathlon', na=True), 'CATEGORY_EVENT'] = 'Pentathlon'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Triathlon', na=True), 'CATEGORY_EVENT'] = 'Triathlon'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Decathlon', na=True), 'CATEGORY_EVENT'] = 'Decathlon'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Marathon', na=True), 'CATEGORY_EVENT'] = 'Marathon'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'4 x 100m', na=True), 'CATEGORY_EVENT'] = 'Relay'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'4 x 400m', na=True), 'CATEGORY_EVENT'] = 'Relay'




In [756]:
# Convert to OLD SCHEMA

#SGP_athletes['TEAM'] = 'Singapore'
#SGP_athletes['SEED'] = ''
#SGP_athletes['LANE'] = ''
#SGP_athletes['DIVISION'] = ''
#SGP_athletes['AGE'] = ''
#SGP_athletes['UNIQUE_ID'] = ''
#SGP_athletes['ATHLETE_ID'] = ''
#SGP_athletes['TIMESTAMP'] = ''
#SGP_athletes['TAG_ID'] = ''
#SGP_athletes['POINTS'] = ''
#SGP_athletes['GROUP'] = ''
#SGP_athletes['FREE_FIELD']=''
#SGP_athletes['FREE_FIELD2']=''
#SGP_athletes['FREE_FIELD3']=''


In [ ]:
# Convert to NEW SCHEMA

SGP_athletes['LAST_NAME'] = ''
SGP_athletes['FIRST_NAME'] = ''
SGP_athletes['OTHER_NAME'] = ''
SGP_athletes['TEAM'] = 'Singapore'
SGP_athletes['SEED'] = ''
SGP_athletes['LANE'] = ''
SGP_athletes['DIVISION'] = ''
SGP_athletes['AGE'] = ''
SGP_athletes['UNIQUE_ID'] = ''
SGP_athletes['ATHLETE_ID'] = ''
SGP_athletes['TIMESTAMP'] = ''
SGP_athletes['TAG_ID'] = ''
SGP_athletes['POINTS'] = ''
SGP_athletes['GROUP'] = ''
SGP_athletes['VENUE']=''
SGP_athletes['SUB_EVENT']=''
SGP_athletes['SESSION']=''
SGP_athletes['EVENT_CLASS']=''
SGP_athletes['SUB_EVENT']=''
SGP_athletes['DISTANCE']=''
SGP_athletes['RX_TIME']=''
SGP_athletes['CATEGORY_EVENT']=''


In [757]:
SGP_athletes

,Unnamed: 0,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,...,AGE,UNIQUE_ID,ATHLETE_ID,TIMESTAMP,TAG_ID,POINTS,GROUP,FREE_FIELD,FREE_FIELD2,FREE_FIELD3
0,102,4.0,Jun Jie Calvin Quek,SGP,53.19,NaN,UAE Athletics Grand Prix,2024,,400m Hurdles,...,,,,,,,,,,
1,31,2.0,Chui Ling Goh,SGP,17:33.73,NaN,11. Münchner Abendsportfest /MUC HochschulMS,2024,,5000m,...,,,,,,,,,,
2,121,4.0,Chen Xiang Ang,SGP,8.01,NaN,11th Asian Indoor Athletics Championships,2024,17 February,60m Hurdles,...,,,,,,,,,,
3,112,8.0,Jun Jie Calvin Quek,SGP,52.05,NaN,11th Kinami Michitaka Memorial Athletics Meet,2024,,400m Hurdles,...,,,,,,,,,,
4,482,4.0,James Ethan Kai Meng Ang,SGP,52.55,NaN,11th Para Athletics World Championships,2024,22 May,400m,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,127,3.0,Jun Jie Calvin Quek,SGP,56.95,NaN,XXXIII Qosanov Memorial,2024,22 June,400m Hurdles,...,,,,,,,,,,
359,148,4.0,Kangli Emery Conrad,SGP,7.22,NaN,XXXIII Qosanov Memorial,2024,22 June,Long Jump,...,,,,,,,,,,
360,320,5.0,Tia Louise Rozario,SGP,5.71,NaN,XXXIII Qosanov Memorial,2024,22 June,Long Jump,...,,,,,,,,,,
361,330,5.0,Tia Louise Rozario,SGP,12.45,NaN,XXXIII Qosanov Memorial,2024,23 June,Triple Jump,...,,,,,,,,,,


In [758]:
df = SGP_athletes.drop(['VENUE', 'YEAR', 'RX_TIME'], axis=1)
df['DATE']='2024'

In [759]:
df

,Unnamed: 0,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,DATE,EVENT,GENDER,...,AGE,UNIQUE_ID,ATHLETE_ID,TIMESTAMP,TAG_ID,POINTS,GROUP,FREE_FIELD,FREE_FIELD2,FREE_FIELD3
0,102,4.0,Jun Jie Calvin Quek,SGP,53.19,NaN,UAE Athletics Grand Prix,2024,400m Hurdles,Male,...,,,,,,,,,,
1,31,2.0,Chui Ling Goh,SGP,17:33.73,NaN,11. Münchner Abendsportfest /MUC HochschulMS,2024,5000m,Female,...,,,,,,,,,,
2,121,4.0,Chen Xiang Ang,SGP,8.01,NaN,11th Asian Indoor Athletics Championships,2024,60m Hurdles,Male,...,,,,,,,,,,
3,112,8.0,Jun Jie Calvin Quek,SGP,52.05,NaN,11th Kinami Michitaka Memorial Athletics Meet,2024,400m Hurdles,Male,...,,,,,,,,,,
4,482,4.0,James Ethan Kai Meng Ang,SGP,52.55,NaN,11th Para Athletics World Championships,2024,400m,Male,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,127,3.0,Jun Jie Calvin Quek,SGP,56.95,NaN,XXXIII Qosanov Memorial,2024,400m Hurdles,Male,...,,,,,,,,,,
359,148,4.0,Kangli Emery Conrad,SGP,7.22,NaN,XXXIII Qosanov Memorial,2024,Long Jump,Male,...,,,,,,,,,,
360,320,5.0,Tia Louise Rozario,SGP,5.71,NaN,XXXIII Qosanov Memorial,2024,Long Jump,Female,...,,,,,,,,,,
361,330,5.0,Tia Louise Rozario,SGP,12.45,NaN,XXXIII Qosanov Memorial,2024,Triple Jump,Female,...,,,,,,,,,,


In [760]:
# Re-order columns

df = df.reindex(columns= ['RANK', 'TAG_ID', 'NAME', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION', 'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 
                                      'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'COUNTRY', 'DICT_RESULTS', 'DATE', 'COMPETITION', 'REGION', 'DOB','GROUP', 'CATEGORY_EVENT', 
                                      'ATHLETE_ID', 'SOURCE', 'REMARKS', 'TIMESTAMP', 'FREE_FIELD', 'FREE_FIELD2', 'FREE_FIELD3'])


In [51]:
df

NameError: name 'df' is not defined

In [762]:
df.columns

Index(['RANK', 'TAG_ID', 'NAME', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION',
       'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 'POINTS', 'AGE',
       'GENDER', 'UNIQUE_ID', 'COUNTRY', 'DICT_RESULTS', 'DATE', 'COMPETITION',
       'REGION', 'DOB', 'GROUP', 'CATEGORY_EVENT', 'ATHLETE_ID', 'SOURCE',
       'REMARKS', 'TIMESTAMP', 'FREE_FIELD', 'FREE_FIELD2', 'FREE_FIELD3'],
      dtype='object')

In [763]:
# Remove special characters
    
for col in df.columns:
    df[col] = df[col].astype(str)
    df[col] = df[col].str.replace('\xa0', ' ', regex=True)
    df[col] = df[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    df[col] = df[col].str.replace('\r', ' ', regex=True)
    df[col] = df[col].str.replace('\n', ' ', regex=True)
    df[col] = df[col].str.strip()


    
    

In [764]:
df.reset_index(inplace=True)

In [50]:
# Check file before saving

ct = dt.now()
gender_pattern = 'Male|Female|Mixed'
year_pattern = '2025'
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
df['YEAR'] = df['YEAR'].astype(str)
df['DATE'] = df['DATE'].astype(str)
df['DOB'] = df['DOB'].astype(str)
#df['event_date_dt'] = pd.to_datetime(df['event_date'], format='mixed', dayfirst=False, utc=True)


for index, row in df.iterrows():
        
    assert re.search(gender_pattern, row['GENDER']) is not None   # check format of gender input
    assert row['NAME'] != None
    assert len(row['NAME'])>5
    assert row['DATE'] != None
    assert row['EVENT'] != None 
    assert row['COMPETITION'] != None 
    assert (row['REGION'] == 'Local' or row['REGION'] == 'International')
    assert row['DATE'] is not None
    assert row['CATEGORY_EVENT'] is not None
    assert re.search('\)|\(', row['EVENT']) is None  # make sure there are no brackets in the event column
    assert len(row['DATE'])>3
    assert row['NATIONALITY']=='SGP'
    assert re.search(year_pattern, row['YEAR']) is not None   # check format of year
    
    # Check column locations before loading

    assert len(df.columns) == 40
    assert df.columns.get_loc('RESULT') == 8
    assert df.columns.get_loc('AGE') == 17
    assert df.columns.get_loc('WIND') == 12
    assert df.columns.get_loc('COMPETITION')==24
    
   

    # Set timestamp

    df.loc[:,'TIMESTAMP'] = ct


print('PASSED')

df

NameError: name 'df' is not defined

In [765]:
df.to_csv('SG_athletes_international_2024.csv', encoding='utf-8')

# Process missing data extracted from SAA database

In [237]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/External events/')

athletes = pd.read_csv("ALL_external_data_from_SAA_database_Jan 2011_to_Dec_24.csv")

In [238]:
athletes

,Date,Event,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any"
0,19 Dec 2024,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-
1,19 Dec 2024,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-
2,07 Dec 2024,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-
3,07 Dec 2024,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-
4,07 Dec 2024,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-
...,...,...,...,...,...,...,...,...,...,...
6535,16 Apr 2011,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne"
6536,27 Mar 2011,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee..."
6537,27 Mar 2011,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne"
6538,11 Mar 2011,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand"


In [239]:
# Remove special characters
    
for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()



In [240]:
athletes['REGION'] = 'International' 
athletes['SOURCE'] = 'SAA Database'



In [241]:
# convert Date column into datetime

athletes['Date']=pd.to_datetime(athletes['Date'], format='mixed')

athletes['Date']

0      2024-12-19
1      2024-12-19
2      2024-12-07
3      2024-12-07
4      2024-12-07
          ...    
6535   2011-04-16
6536   2011-03-27
6537   2011-03-27
6538   2011-03-11
6539   2011-02-20
Name: Date, Length: 6540, dtype: datetime64[ns]

In [242]:
# extract day and month only

athletes['date_extract']=athletes['Date'].dt.strftime('%d-%m')
athletes['YEAR']=athletes['Date'].dt.strftime('%Y')

In [243]:
athletes

,Date,Event,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any",REGION,SOURCE,date_extract,YEAR
0,2024-12-19,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-,International,SAA Database,19-12,2024
1,2024-12-19,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-,International,SAA Database,19-12,2024
2,2024-12-07,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024
3,2024-12-07,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024
4,2024-12-07,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-,International,SAA Database,07-12,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2011-04-16,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne",International,SAA Database,16-04,2011
6536,2011-03-27,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee...",International,SAA Database,27-03,2011
6537,2011-03-27,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne",International,SAA Database,27-03,2011
6538,2011-03-11,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand",International,SAA Database,11-03,2011


In [244]:
mask = athletes['Event'].str.contains(r'Men', na=True)
athletes.loc[mask, 'GENDER'] = 'Male'

mask = athletes['Event'].str.contains(r'Women', na=True)
athletes.loc[mask, 'GENDER'] = 'Female'


In [245]:
athletes

,Date,Event,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any",REGION,SOURCE,date_extract,YEAR,GENDER
0,2024-12-19,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-,International,SAA Database,19-12,2024,Male
1,2024-12-19,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-,International,SAA Database,19-12,2024,Male
2,2024-12-07,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024,Male
3,2024-12-07,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024,Male
4,2024-12-07,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-,International,SAA Database,07-12,2024,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2011-04-16,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne",International,SAA Database,16-04,2011,Male
6536,2011-03-27,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee...",International,SAA Database,27-03,2011,Female
6537,2011-03-27,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne",International,SAA Database,27-03,2011,Male
6538,2011-03-11,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand",International,SAA Database,11-03,2011,Male


In [246]:
athletes['COMPETITION'] = athletes['Competition'].str[16:]  # Remove 'International -' from string

In [247]:
athletes

,Date,Event,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any",REGION,SOURCE,date_extract,YEAR,GENDER,COMPETITION
0,2024-12-19,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-,International,SAA Database,19-12,2024,Male,Strive (Australia)
1,2024-12-19,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-,International,SAA Database,19-12,2024,Male,Vic Milers (Australia)
2,2024-12-07,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)"
3,2024-12-07,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)"
4,2024-12-07,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-,International,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2011-04-16,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne",International,SAA Database,16-04,2011,Male,Australia Athletics Cships
6536,2011-03-27,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee...",International,SAA Database,27-03,2011,Female,Queensland Open & AWD Cships
6537,2011-03-27,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne",International,SAA Database,27-03,2011,Male,Athletic Victoria Throwers Meet #6
6538,2011-03-11,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand",International,SAA Database,11-03,2011,Male,Thai Veterans Athletics Cships


In [248]:
mask = athletes['Info, if any'].str.contains(r'Final', na=True)
athletes.loc[mask, 'STAGE'] = 'Final'

mask = athletes['Info, if any'].str.contains(r'Prelim', na=True)
athletes.loc[mask, 'STAGE'] = 'Prelim'

mask = athletes['Info, if any'].str.contains(r'Semifinal', na=True)
athletes.loc[mask, 'STAGE'] = 'Semi'

mask = athletes['Info, if any'].str.contains(r'Semi', na=True)
athletes.loc[mask, 'STAGE'] = 'Semi'


mask = athletes['Info, if any'].str.contains(r'Heats', na=True)
athletes.loc[mask, 'STAGE'] = 'Heats'


In [249]:
mask = athletes['Info, if any'].str.contains(r'U20', na=True)
athletes.loc[mask, 'DIVISION'] = 'U20'

mask = athletes['Info, if any'].str.contains(r'U18', na=True)
athletes.loc[mask, 'DIVISION'] = 'U18'


In [250]:
'''

mask = athletes['Info, if any'].str.contains(r'PB', na=True)
athletes.loc[mask, 'REMARKS'] = 'PB'

mask = athletes['Info, if any'].str.contains(r'SB', na=True)
athletes.loc[mask, 'REMARKS'] = 'SB'

mask = athletes['Info, if any'].str.contains(r'CR', na=True)
athletes.loc[mask, 'REMARKS'] = 'CR'

mask = athletes['Info, if any'].str.contains(r'NR', na=True)
athletes.loc[mask, 'REMARKS'] = 'NR'

mask = athletes['Info, if any'].str.contains(r'R1', na=True)
athletes.loc[mask, 'REMARKS'] = 'R1'

'''


"\n\nmask = athletes['Info, if any'].str.contains(r'PB', na=True)\nathletes.loc[mask, 'REMARKS'] = 'PB'\n\nmask = athletes['Info, if any'].str.contains(r'SB', na=True)\nathletes.loc[mask, 'REMARKS'] = 'SB'\n\nmask = athletes['Info, if any'].str.contains(r'CR', na=True)\nathletes.loc[mask, 'REMARKS'] = 'CR'\n\nmask = athletes['Info, if any'].str.contains(r'NR', na=True)\nathletes.loc[mask, 'REMARKS'] = 'NR'\n\nmask = athletes['Info, if any'].str.contains(r'R1', na=True)\nathletes.loc[mask, 'REMARKS'] = 'R1'\n\n"

In [251]:
athletes.rename(columns={'Event': 'EVENT'}, inplace=True)


In [252]:
# Running

mask = athletes['EVENT'].str.contains(r'50 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '50m'
mask = athletes['EVENT'].str.contains(r'60 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '60m'
mask = athletes['EVENT'].str.contains(r'80 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m'
mask = athletes['EVENT'].str.contains(r'100 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'^100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'200 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'^200m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'200\sMeter', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'300 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '300m'
mask = athletes['EVENT'].str.contains(r'400 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'

mask = athletes['EVENT'].str.contains(r'^400\sMeter$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'


mask = athletes['EVENT'].str.contains(r'600 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '600m'
mask = athletes['EVENT'].str.contains(r'800 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'^800m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'1500 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'^1500m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'3000 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = athletes['EVENT'].str.contains(r'3000m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = athletes['EVENT'].str.contains(r'5000 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'5000m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'10000 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'
mask = athletes['EVENT'].str.contains(r'^10000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'10 Kilometer Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'


mask = athletes['EVENT'].str.contains(r'1 Mile Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1 mile'
mask = athletes['EVENT'].str.contains(r'Mile', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1 mile'


mask = athletes['EVENT'].str.contains(r'10\,000m', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'



# Hurdles

mask = athletes['EVENT'].str.contains(r'^80m\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'
mask = athletes['EVENT'].str.contains(r'^80m\shurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'
mask = athletes['EVENT'].str.contains(r'^80\sMeter\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m Hurdles'
mask = athletes['EVENT'].str.contains(r'^100m\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = athletes['EVENT'].str.contains(r'^100\sMeter\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m hurdles'
mask = athletes['EVENT'].str.contains(r'100\sMeter\sHurdles\s\(0\.838m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'
mask = athletes['EVENT'].str.contains(r'100m\sHurdles\s\(0\.838m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m Hurdles'

mask = athletes['EVENT'].str.contains(r'110\sMeter\sHurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = athletes['EVENT'].str.contains(r'^110m\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'


#mask = athletes['EVENT'].str.contains(r'110\sMeter\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
#mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'

mask = athletes['EVENT'].str.contains(r'110\sMeter\sHurdles\s\(1\.067m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(1\.067m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'
mask = athletes['EVENT'].str.contains(r'110\sMeter\sHurdles\sOpen', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m Hurdles'



mask = athletes['EVENT'].str.contains(r'^200m\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m Hurdles'
mask = athletes['EVENT'].str.contains(r'200 Meter Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m Hurdles'
mask = athletes['EVENT'].str.contains(r'200m Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m Hurdles'
mask = athletes['EVENT'].str.contains(r'200m\sHurdles\s\(0\.762m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = ' '

mask = athletes['EVENT'].str.contains(r'400m\sHurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0.840m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = ' '

mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


mask = athletes['EVENT'].str.contains(r'400m\sHurdles\sOpen', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\sOpen', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'

mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\sOPEN', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\s\(0\.914m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'
mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0\.914m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'

mask = athletes['EVENT'].str.contains(r'^400\sMeter\sHurdles\s\(0\.762m\)$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'
mask = athletes['EVENT'].str.contains(r'^400m\sHurdles\s\(0\.762m\)$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m Hurdles'


# Throws

mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(600g\)', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = athletes['EVENT'].str.contains(r'^Javelin\sThrow$', na=True, regex=True)  # exact match for string
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g', na=True, regex=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(800g\)', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\sOpen', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\sOPEN', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'
mask = athletes['EVENT'].str.contains(r'Javelin\sThrow', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin Throw'



mask = athletes['EVENT'].str.contains(r'^Shot\sPut$', na=True, regex=True) # there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

mask = (athletes['EVENT'].str.contains(r'Shot\sPut\s\(4\.00kg\)', na=True, regex=True) & (athletes['GENDER']=='Female'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = (athletes['EVENT'].str.contains(r'Shot\sPut\s\(4kg\)', na=True, regex=True) & (athletes['GENDER']=='Female'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'


mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s4kg\sOpen', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
#mask = athletes['EVENT'].str.contains(r'Men\sShot\sPut\s4kg\sOPEN', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'
mask = athletes['EVENT'].str.contains(r'Shot\sPut\sOPEN', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'


mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s\(4kg\)', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = athletes['EVENT'].str.contains(r'Shot\sPut\s\(7\.26kg\)', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = athletes['EVENT'].str.contains(r'Shot\sPut\s7\.26kg\sOpen', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'

mask = athletes['EVENT'].str.contains(r'Shot\sPut\sOpen', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'
mask = athletes['EVENT'].str.contains(r'^Shot\sput$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot Put'


#mask = athletes['EVENT'].str.contains(r'Shot Put Masters', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'


mask = athletes['EVENT'].str.contains(r'^Hammer\sThrow$', na=True)  # there are some characters after Throw
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'

mask = (athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(4kg\)', na=True) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'
mask = athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(7\.26kg\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer Throw'

#mask = athletes['EVENT'].str.contains(r'^Discus\sThrow    $', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'
mask = athletes['EVENT'].str.contains(r'Discus\sThrow', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = athletes['EVENT'].str.contains(r'Discus\sthrow', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'


mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'

mask = ((athletes['EVENT'].str.contains(r'Discus\s\(1\.00kg\)', na=True))  & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'



mask = athletes['EVENT'].str.contains(r'Women\sDiscus\sThrow\s\(1kg\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'



mask = athletes['EVENT'].str.contains(r'Discus\sThrow\s\(2kg\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = athletes['EVENT'].str.contains(r'Discus\sThrow\sOpen', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'
mask = athletes['EVENT'].str.contains(r'Discus\sThrow\sOPEN', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus Throw'



# Jumps

mask = athletes['EVENT'].str.contains(r'High Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'

mask = athletes['EVENT'].str.contains(r'Long\sJump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Open', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Trial', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'


mask = athletes['EVENT'].str.contains(r'Triple Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'Pole Vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'
mask = athletes['EVENT'].str.contains(r'High jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High Jump'
mask = athletes['EVENT'].str.contains(r'Long jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long Jump'
mask = athletes['EVENT'].str.contains(r'Triple jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple Jump'
mask = athletes['EVENT'].str.contains(r'^Pole\svault$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole Vault'

# Steeplechase

mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = athletes['EVENT'].str.contains(r'2000m Steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = athletes['EVENT'].str.contains(r'3000m S/C', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'
mask = athletes['EVENT'].str.contains(r'3000 Meter Steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'

# Marathon

mask = athletes['EVENT'].str.contains(r'Marathon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Marathon'
mask = athletes['EVENT'].str.contains(r'Half\sMarathon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half marathon'

# Walk

mask = athletes['EVENT'].str.contains(r'1500m Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Racewalk'
mask = athletes['EVENT'].str.contains(r'1500 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Racewalk'
mask = athletes['EVENT'].str.contains(r'3000m Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Racewalk'
mask = athletes['EVENT'].str.contains(r'3000 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Racewalk'
mask = athletes['EVENT'].str.contains(r'5000 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Racewalk'
mask = athletes['EVENT'].str.contains(r'5000m Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Racewalk'
mask = athletes['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m Racewalk'

# Relay

mask = athletes['EVENT'].str.contains(r'4x80m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 80m Relay'
mask = athletes['EVENT'].str.contains(r'^4\sx\s100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m Relay'
mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m Relay'
mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m Relay'
mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m Relay'
mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m Relay'
mask = athletes['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m Relay'
mask = athletes['EVENT'].str.contains(r'4x400 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m Relay'
mask = athletes['EVENT'].str.contains(r'^4\sx\s400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m Relay'

# Decathlon/Heptathlon

mask = athletes['EVENT'].str.contains(r'^Heptathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'^Decathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'
mask = athletes['EVENT'].str.contains(r'Heptathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'Decathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'

# Cross Country

mask = athletes['EVENT'].str.contains(r'10 Kilometer Run (Cross Country)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10km Cross Country'



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_13049/745616938.py:340: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = athletes['EVENT'].str.contains(r'10 Kilometer Run (Cross Country)', na=True)


In [253]:
athletes

,Date,EVENT,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any",REGION,SOURCE,date_extract,YEAR,GENDER,COMPETITION,STAGE,DIVISION,MAPPED_EVENT
0,2024-12-19,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-,International,SAA Database,19-12,2024,Male,Strive (Australia),NaN,NaN,800m
1,2024-12-19,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-,International,SAA Database,19-12,2024,Male,Vic Milers (Australia),NaN,NaN,1500m
2,2024-12-07,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,60m
3,2024-12-07,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,200m
4,2024-12-07,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-,International,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,60m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2011-04-16,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne",International,SAA Database,16-04,2011,Male,Australia Athletics Cships,Prelim,NaN,Shot Put
6536,2011-03-27,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee...",International,SAA Database,27-03,2011,Female,Queensland Open & AWD Cships,Final,NaN,10000m
6537,2011-03-27,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne",International,SAA Database,27-03,2011,Male,Athletic Victoria Throwers Meet #6,Final,NaN,Shot Put
6538,2011-03-11,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand",International,SAA Database,11-03,2011,Male,Thai Veterans Athletics Cships,Final,NaN,5000m Racewalk


In [254]:
# Extract ranking

def rank(string):
    
    

    try:
        
        slash = re.search(r'\/', string) 
        
        d1 = re.search(r'^\dpos', string)
        d2 = re.search(r'^\d\dpos', string)
        d3 = re.search(r'^\d\d\d\dpos', string)
        d4 = re.search(r'^\d\d\dpos', string)
        
        print(d1, d2, d3, d4)
        
        print(slash)
        
        if slash:
     
            start = slash.start()-2
    
            end = slash.start()
    
            pos = string[start:end]
        
        elif d1:
            
            pos = string[:1]
            
            print(d1)
            
        elif d2:
            
            pos = string[:2]
            
            print(d2)
            
        elif d3:
            
            pos = string[:4]
            
        elif d4:
            
            pos = string[:3]
            
            
        else:
            
            pos=''
            
    except:
        
        pos = ''
    
    return pos
    

athletes['RANK'] = athletes['Info, if any'].apply(rank)


None None None None
None
None None None None
None
None None None None
None
None None None None
None
None None None None
None
None None None None
None
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(12, 13), match='/'>
None None None None
<re.Match object; span=(12, 13), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(12, 13), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; 

In [6]:
# Extract date

def date(string):
    
    
    try:
        
        slash=re.search(r'2025', string)
        
        end = slash.start()
    
        date = string[:end]
            
    except:
        
        date = ''
    
    return date
    

#athletes['DATE'] = athletes['Date'].apply(date)


In [7]:
athletes

NameError: name 'athletes' is not defined

In [257]:
# Extract heat

def heat(string):
    
    
    try:
        
        heat=re.findall(r'h\d|H\d|h\d\d|H\d\d', string)
        
        heat1=heat[0]
                
            
    except:
        
        heat1 = ''
    
    return heat1
    

athletes['HEAT'] = athletes['Info, if any'].apply(heat)


In [258]:
athletes

,Date,EVENT,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any",...,SOURCE,date_extract,YEAR,GENDER,COMPETITION,STAGE,DIVISION,MAPPED_EVENT,RANK,HEAT
0,2024-12-19,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-,...,SAA Database,19-12,2024,Male,Strive (Australia),NaN,NaN,800m,,
1,2024-12-19,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-,...,SAA Database,19-12,2024,Male,Vic Milers (Australia),NaN,NaN,1500m,,
2,2024-12-07,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-,...,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,60m,,
3,2024-12-07,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-,...,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,200m,,
4,2024-12-07,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-,...,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,60m,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2011-04-16,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne",...,SAA Database,16-04,2011,Male,Australia Athletics Cships,Prelim,NaN,Shot Put,,
6536,2011-03-27,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee...",...,SAA Database,27-03,2011,Female,Queensland Open & AWD Cships,Final,NaN,10000m,,
6537,2011-03-27,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne",...,SAA Database,27-03,2011,Male,Athletic Victoria Throwers Meet #6,Final,NaN,Shot Put,,
6538,2011-03-11,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand",...,SAA Database,11-03,2011,Male,Thai Veterans Athletics Cships,Final,NaN,5000m Racewalk,,


In [5]:
# Extract qualitication

def qual(string):
    
    
    try:
        
        qual=re.findall(r'\sq|\sQ', string)
        qual1=qual[0]
        
                
            
    except:
        
        qual1 = ''
    
    return qual1
    

athletes['QUALIFICATION'] = athletes['Info, if any'].apply(qual)


NameError: name 'athletes' is not defined

In [260]:
# Extract event class

def event_class(string):
    
    
    try:
        
        left=re.search(r'\(', string)
        right=re.search(r'\)', string)
        
        
        type = string[left.start()+1:right.start()]
        
                 
    except:
        
        type = ''
    
    return type
    

athletes['EVENT_CLASS'] = athletes['EVENT'].apply(event_class)


In [261]:
athletes

,Date,EVENT,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any",...,YEAR,GENDER,COMPETITION,STAGE,DIVISION,MAPPED_EVENT,RANK,HEAT,QUALIFICATION,EVENT_CLASS
0,2024-12-19,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-,...,2024,Male,Strive (Australia),NaN,NaN,800m,,,,
1,2024-12-19,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-,...,2024,Male,Vic Milers (Australia),NaN,NaN,1500m,,,,
2,2024-12-07,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-,...,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,60m,,,,
3,2024-12-07,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-,...,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,200m,,,,
4,2024-12-07,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-,...,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,60m,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2011-04-16,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne",...,2011,Male,Australia Athletics Cships,Prelim,NaN,Shot Put,,,,7.26kg
6536,2011-03-27,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee...",...,2011,Female,Queensland Open & AWD Cships,Final,NaN,10000m,,,Q,
6537,2011-03-27,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne",...,2011,Male,Athletic Victoria Throwers Meet #6,Final,NaN,Shot Put,,,,7.26kg
6538,2011-03-11,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand",...,2011,Male,Thai Veterans Athletics Cships,Final,NaN,5000m Racewalk,,,,


In [262]:
athletes.to_csv("SAA_external_database.csv", encoding="utf-8")

In [263]:
athletes = athletes.rename(columns={'Info, if any': 'REMARKS', 'Wind m/s': "WIND", 'Age': "AGE", 'Result': "RESULT", 'date_extract': "DATE", 'Year D.O.B.': "DOB"})

In [264]:
athletes = athletes.drop(['Date', 'Competition', 'EVENT'], axis=1)

In [265]:
athletes = athletes.rename(columns={'MAPPED_EVENT': "EVENT", 'Name': "NAME", 'Team': "TEAM"})

In [266]:
athletes = athletes[athletes['YEAR']=='2023']

In [267]:
athletes

,NAME,AGE,TEAM,RESULT,WIND,DOB,REMARKS,REGION,SOURCE,DATE,YEAR,GENDER,COMPETITION,STAGE,DIVISION,EVENT,RANK,HEAT,QUALIFICATION,EVENT_CLASS
530,"NORHISHAM, JAMIE EL-REDHA ANG",19,Singapore Sports School,11.01,-0.2,2004,Heats 1/8pos q,International,SAA Database,20-12,2023,Male,SAECA-BJSS Athletics Championship,Heats,NaN,100m,1,,q,
531,"NORHISHAM, JAMIE EL-REDHA ANG",19,Singapore Sports School,11.08,-0.1,2004,Final 8/8pos,International,SAA Database,20-12,2023,Male,SAECA-BJSS Athletics Championship,Final,NaN,100m,8,,,
532,"MUTHUKUMARAN, RAAM KUMAR",19,Singapore Sports School,3:20.75,-,2004,Final 1/8pos,International,SAA Database,20-12,2023,Male,SAECA-BJSS Athletics Championship,Final,NaN,4 x 400m Relay,1,,,
533,"NORHISHAM, JAMIE EL-REDHA ANG",19,Singapore Sports School,22.32,-0.3,2004,Final 3/8pos,International,SAA Database,19-12,2023,Male,SAECA-BJSS Athletics Championship,Final,NaN,200m,3,,,
534,"YUSRAN, NIQ RIFDI RAFIQI",16,Singapore Sports School,55.05,-,2007,Heats 4/8pos,International,SAA Database,19-12,2023,Male,SAECA-BJSS Athletics Championship,Heats,NaN,400m,4,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2114,"ROZARIO, TIA LOUISE",23,PRINCETON,5.52,Indoor,2000,"5/8pos, sanctioned",International,SAA Database,28-01,2023,Female,Harvard-Yale-Princeton,NaN,NaN,Long Jump,,,,
2115,"KAM, KAMPTON",22,INDIVIDUAL,2.08,-,2001,"Sanctioned,1pos",International,SAA Database,21-01,2023,Male,Wesley A Brown Invitational,NaN,NaN,High Jump,,,,
2116,"PHUA, JASMIN",22,Individual,42.83,-,2000,-,International,SAA Database,20-01,2023,Female,Strive Season Program B,NaN,NaN,Discus Throw,,,,1kg
2117,"KAM, KAMPTON",22,Individual,2.06,-,2000,"Sanctioned,1pos",International,SAA Database,14-01,2023,Male,Penn 10-Team Select,NaN,NaN,High Jump,,,,


In [268]:
athletes.to_csv("athletes_2023.csv", encoding='utf-8')

In [269]:
# Map general event category 

athletes.loc[athletes['EVENT'].str.contains(r'100m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'400m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'600m', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes.loc[athletes['EVENT'].str.contains(r'60m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'200m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'50m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'80m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'300m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'3000m', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes.loc[athletes['EVENT'].str.contains(r'Mile', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes.loc[athletes['EVENT'].str.contains(r'mile', na=True), 'CATEGORY_EVENT'] = 'Mid'

athletes.loc[athletes['EVENT'].str.contains(r'5000m', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes.loc[athletes['EVENT'].str.contains(r'2000m', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes.loc[athletes['EVENT'].str.contains(r'800m', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes.loc[athletes['EVENT'].str.contains(r'1500m', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes.loc[athletes['EVENT'].str.contains(r'10,000m', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes.loc[athletes['EVENT'].str.contains(r'10000m', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes.loc[athletes['EVENT'].str.contains(r'5km', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes.loc[athletes['EVENT'].str.contains(r'10km', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes.loc[athletes['EVENT'].str.contains(r'Cross Country', na=True), 'CATEGORY_EVENT'] = 'Cross Country'


# Override values

athletes.loc[athletes['EVENT'].str.contains(r'Discus', na=True), 'CATEGORY_EVENT'] = 'Throw'
athletes.loc[athletes['EVENT'].str.contains(r'Javelin', na=True), 'CATEGORY_EVENT'] = 'Throw'
athletes.loc[athletes['EVENT'].str.contains(r'Shot', na=True), 'CATEGORY_EVENT'] = 'Throw'
athletes.loc[athletes['EVENT'].str.contains(r'Vault', na=True), 'CATEGORY_EVENT'] = 'Jump'
athletes.loc[athletes['EVENT'].str.contains(r'vault', na=True), 'CATEGORY_EVENT'] = 'Jump'

athletes.loc[athletes['EVENT'].str.contains(r'Throw', na=True), 'CATEGORY_EVENT'] = 'Throw'
athletes.loc[athletes['EVENT'].str.contains(r'Jump', na=True), 'CATEGORY_EVENT'] = 'Jump'
athletes.loc[athletes['EVENT'].str.contains(r'jump', na=True), 'CATEGORY_EVENT'] = 'Jump'

athletes.loc[athletes['EVENT'].str.contains(r'Relay', na=True), 'CATEGORY_EVENT'] = 'Relay'
athletes.loc[athletes['EVENT'].str.contains(r'steeple', na=True), 'CATEGORY_EVENT'] = 'Steeple'
athletes.loc[athletes['EVENT'].str.contains(r'Walk', na=True), 'CATEGORY_EVENT'] = 'Walk'
athletes.loc[athletes['EVENT'].str.contains(r'Hurdles', na=True), 'CATEGORY_EVENT'] = 'Hurdles'
athletes.loc[athletes['EVENT'].str.contains(r'hurdles', na=True), 'CATEGORY_EVENT'] = 'Hurdles'

athletes.loc[athletes['EVENT'].str.contains(r'Pentathlon', na=True), 'CATEGORY_EVENT'] = 'Pentathlon'
athletes.loc[athletes['EVENT'].str.contains(r'Triathlon', na=True), 'CATEGORY_EVENT'] = 'Triathlon'
athletes.loc[athletes['EVENT'].str.contains(r'Decathlon', na=True), 'CATEGORY_EVENT'] = 'Decathlon'
athletes.loc[athletes['EVENT'].str.contains(r'Marathon', na=True), 'CATEGORY_EVENT'] = 'Marathon'
athletes.loc[athletes['EVENT'].str.contains(r'marathon', na=True), 'CATEGORY_EVENT'] = 'Marathon'
athletes.loc[athletes['EVENT'].str.contains(r'4 x 100m', na=True), 'CATEGORY_EVENT'] = 'Relay'
athletes.loc[athletes['EVENT'].str.contains(r'4 x 400m', na=True), 'CATEGORY_EVENT'] = 'Relay'
athletes.loc[athletes['EVENT'].str.contains(r'Steeplechase', na=True), 'CATEGORY_EVENT'] = 'Steeple'




In [270]:
# Extract date

def time(string):
    
    
    try:
        
        slash=re.search(r'AM|PM', string)
        
        end = slash.start()
    
        time = string[:end]
            
    except:
        
        time = ''
    
    return time
    

#athletes['RESULT_CLEANED'] = athletes['RESULT'].apply(time)


In [271]:
athletes.columns

Index(['NAME', 'AGE', 'TEAM', 'RESULT', 'WIND', 'DOB', 'REMARKS', 'REGION',
       'SOURCE', 'DATE', 'YEAR', 'GENDER', 'COMPETITION', 'STAGE', 'DIVISION',
       'EVENT', 'RANK', 'HEAT', 'QUALIFICATION', 'EVENT_CLASS',
       'CATEGORY_EVENT'],
      dtype='object')

In [272]:
# Convert to NEW SCHEMA

athletes['LAST_NAME'] = ''
athletes['FIRST_NAME'] = ''
athletes['OTHER_NAME'] = ''
athletes['SEED'] = ''
athletes['LANE'] = ''
athletes['ATHLETE_ID'] = ''
athletes['TIMESTAMP'] = ''
athletes['TAG_ID'] = ''
athletes['POINTS'] = ''
athletes['GROUP'] = ''
athletes['SESSION']=''
athletes['DISTANCE']=''
athletes['HOST_CITY']=''
athletes['SUB_EVENT']=''
athletes['DICT_RESULTS']=''
athletes['RX_TIME']=''






athletes = athletes.reindex(columns= ['FIRST_NAME', 'LAST_NAME', 'OTHER_NAME', 'NAME', 'RANK', 'TAG_ID', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION',
                                        'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'NATIONALITY',
                                        'DICT_RESULTS', 'YEAR', 'DATE', 'COMPETITION', 'REGION', 'DOB', 'GROUP', 'CATEGORY_EVENT', 'ATHLETE_ID',
                                        'SOURCE', 'REMARKS', 'TIMESTAMP', 'VENUE', 'SUB_EVENT', 'SESSION', 'EVENT_CLASS', 'DISTANCE', 'HOST_CITY', 'RX_TIME'])


In [273]:
Marathon_2023 = athletes[athletes['EVENT']=='Marathon']

In [275]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/SAA Database/')


Marathon_2023.to_csv("SAA_database_marathon_2023.csv", encoding='utf-8')

In [56]:
athletes

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
0,,,,NaN,,,NaN,,1:53.51,,...,SAA Database,NaN,,NaN,,,,,,
1,,,,NaN,,,NaN,,4:01.49,,...,SAA Database,NaN,,NaN,,,,,,
2,,,,NaN,,,NaN,,6.84,,...,SAA Database,NaN,,NaN,,,,,,
3,,,,NaN,,,NaN,,21.68,,...,SAA Database,NaN,,NaN,,,,,,
4,,,,NaN,,,NaN,,6.97,,...,SAA Database,NaN,,NaN,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,,,,NaN,,,NaN,,14.91,,...,SAA Database,NaN,,NaN,,,7.26kg,,,
6536,,,,NaN,,,NaN,,37:15.87,Q,...,SAA Database,NaN,,NaN,,,,,,
6537,,,,NaN,,,NaN,,15.32,,...,SAA Database,NaN,,NaN,,,7.26kg,,,
6538,,,,NaN,,,NaN,,26:53,,...,SAA Database,NaN,,NaN,,,,,,


# Check googe sheets downloand and retrieval

In [122]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Google Sheets/2025/Aug/')

google = pd.read_csv("SAA Competitions - Competitions.csv")


In [123]:
google

,Competition Name,Country,Competition Date,Athlete Name,Event,Result,Wind,Gender,Comments,Uploads
0,"Queensland 10,000m Championships",Australia,09/08/2025,Shaun Goh,10000m,31:02.40,NaN,Male,https://worldathletics.org/competition/calenda...,https://drive.google.com/file/d/196cOw1tYQUkRY...
1,"Queensland 10,000m Championships",Australia,09/08/2025,Vanessa Lee,10000m,36:15.67,NaN,Female,https://worldathletics.org/competition/calenda...,https://drive.google.com/file/d/196cOw1tYQUkRY...
2,7th Tokai Sprint Games,Japan,17/08/2025,Calvin Quek,100m,10.72,0.4,Male,https://worldathletics.org/competition/calenda...,NaN
3,7th Tokai Sprint Games,Japan,17/08/2025,Praharsh Ryan,100m,10.70,0.0,Male,https://worldathletics.org/competition/calenda...,NaN
4,7th Tokai Sprint Games,Japan,17/08/2025,Praharsh Ryan,100m,10.70,0.1,Male,https://worldathletics.org/competition/calenda...,NaN
5,2025 ICAAK Games,Japan,12/08/2025,Calvin Quek,400m Hurdles (0.914m),50.24,NaN,Male,https://worldathletics.org/competition/calenda...,https://drive.google.com/file/d/13TsWlLosDFnOu...
6,20th Twilight Games,Japan,20/08/2025,Calvin Quek,400m Hurdles (0.914m),49.75,NaN,Male,NaN,NaN


In [125]:
google['Result']=google['Result'].astype(str)

In [126]:
google

,Competition Name,Country,Competition Date,Athlete Name,Event,Result,Wind,Gender,Comments,Uploads
0,"Queensland 10,000m Championships",Australia,09/08/2025,Shaun Goh,10000m,31:02.40,NaN,Male,https://worldathletics.org/competition/calenda...,https://drive.google.com/file/d/196cOw1tYQUkRY...
1,"Queensland 10,000m Championships",Australia,09/08/2025,Vanessa Lee,10000m,36:15.67,NaN,Female,https://worldathletics.org/competition/calenda...,https://drive.google.com/file/d/196cOw1tYQUkRY...
2,7th Tokai Sprint Games,Japan,17/08/2025,Calvin Quek,100m,10.72,0.4,Male,https://worldathletics.org/competition/calenda...,NaN
3,7th Tokai Sprint Games,Japan,17/08/2025,Praharsh Ryan,100m,10.70,0.0,Male,https://worldathletics.org/competition/calenda...,NaN
4,7th Tokai Sprint Games,Japan,17/08/2025,Praharsh Ryan,100m,10.70,0.1,Male,https://worldathletics.org/competition/calenda...,NaN
5,2025 ICAAK Games,Japan,12/08/2025,Calvin Quek,400m Hurdles (0.914m),50.24,NaN,Male,https://worldathletics.org/competition/calenda...,https://drive.google.com/file/d/13TsWlLosDFnOu...
6,20th Twilight Games,Japan,20/08/2025,Calvin Quek,400m Hurdles (0.914m),49.75,NaN,Male,NaN,NaN
